In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
fileName = "xid-92685698_2.txt" 
df = pd.read_csv(fileName, delim_whitespace=True) 
df.head()

,YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,2023,1,1,0,26,999,99.0,99.0,1.12,12.50,7.69,269,9999.0,999.0,15.3,999.0,99.0,99.0
1,2023,1,1,0,56,999,99.0,99.0,1.16,11.76,7.34,272,9999.0,999.0,15.3,999.0,99.0,99.0
2,2023,1,1,1,26,999,99.0,99.0,1.20,12.50,7.08,268,9999.0,999.0,15.3,999.0,99.0,99.0
3,2023,1,1,1,56,999,99.0,99.0,1.25,10.53,7.33,268,9999.0,999.0,15.4,999.0,99.0,99.0
4,2023,1,1,2,26,999,99.0,99.0,1.27,10.53,7.57,268,9999.0,999.0,15.5,999.0,99.0,99.0


In [3]:
df_w = df[["DPD","APD","MWD","WTMP","WVHT"]]
df_w.head()

,DPD,APD,MWD,WTMP,WVHT
0,12.50,7.69,269,15.3,1.12
1,11.76,7.34,272,15.3,1.16
2,12.50,7.08,268,15.3,1.20
3,10.53,7.33,268,15.4,1.25
4,10.53,7.57,268,15.5,1.27


In [4]:
df_w1 = df_w.drop(df_w[df_w['DPD'] == 99.0].index)
df_w2 = df_w1.drop(df_w1[df_w1['APD'] == 99.0].index)
df_w3 = df_w2.drop(df_w2[df_w2['MWD'] == 999.0].index)
df_w4 = df_w3.drop(df_w3[df_w3['WTMP'] == 99.0].index)
df_w5 = df_w4.drop(df_w4[df_w4['WVHT'] == 99.0].index)

In [16]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
# Mean squared error 
from sklearn.metrics import mean_squared_error

In [18]:
xVal = df_w5[['DPD','APD','WTMP','MWD']].to_numpy()
yVal = np.array(df_w5['WVHT'])

In [19]:
from sklearn.model_selection import train_test_split
X_train_temp, X_temp, y_train_temp, y_temp = train_test_split(xVal, yVal, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5)

### Baseline Model

In [20]:
from sklearn import linear_model
lm= linear_model.LinearRegression()
lm.fit(X_train_temp,y_train_temp)
lm_y1 = lm.predict(X_train_temp)
lm_y2 = lm.predict(X_test)
lm_y3 = lm.predict(X_val)
lm_err = mean_squared_error(lm_y1,y_train_temp)
print(mean_squared_error(lm_y1,y_train_temp))
print(mean_squared_error(lm_y2,y_test))
print(mean_squared_error(lm_y3,y_val))
#print(lm.score(X_train_temp, y_train_temp))

0.19297046969447665
0.20917035013664229
0.20370669222151108


### MLP regressor

In [21]:
regr = MLPRegressor(hidden_layer_sizes=(16,16,16,)).fit(X_train_temp, y_train_temp)

lm_y1 = regr.predict(X_train_temp)
lm_y2 = regr.predict(X_test)
lm_y3 = regr.predict(X_val)

print(mean_squared_error(lm_y1,y_train_temp))
print(mean_squared_error(lm_y2,y_test))
print(mean_squared_error(lm_y3,y_val))


0.18248030704168555
0.19618085045793712
0.19518520904966608


Which model performs best?
- Multilayer perceptron regressor performs better.

In [42]:
nodes = [(1,),
         (16,),
         (1,16,),
         (16,16,),
         (16,4,16,),
         (16,16,16,),
         (16,16,16,16,),
         (16,32,24,16,),
         (1,1,1,1,1,),
         (100,100,100,100,100,)]
for i in nodes:
    regr = MLPRegressor(hidden_layer_sizes=i).fit(X_train_temp, y_train_temp)
    lm_y1 = regr.predict(X_train_temp)
    lm_y2 = regr.predict(X_test)
    lm_y3 = regr.predict(X_val)

    print(i,' Train error', mean_squared_error(lm_y1,y_train_temp),"Test error", mean_squared_error(lm_y2,y_test))
   


(1,)  Train error 0.27333979223460575 Test error 0.3083485347562095
(16,)  Train error 0.18253845749023337 Test error 0.18676513239220136
(1, 16)  Train error 0.27329041956324784 Test error 0.3080563956911877
(16, 16)  Train error 0.2125149785728048 Test error 0.22515085878139868
(16, 4, 16)  Train error 0.1736755952089604 Test error 0.182304055832702
(16, 16, 16)  Train error 0.18891629480149988 Test error 0.20244334292199234
(16, 16, 16, 16)  Train error 0.19409742030736846 Test error 0.20621351476236982
(16, 32, 24, 16)  Train error 0.19023650731787323 Test error 0.20673131138043754
(1, 1, 1, 1, 1)  Train error 0.273276356382285 Test error 0.30821169688742134
(100, 100, 100, 100, 100)  Train error 0.18116623884370978 Test error 0.1932817059358181


What conclusions can you come to about how changing layer and node numbers impacts predictive skill?
- When the number of nodes increases, the error decreases. The error also reduces for the same number of nodes with increasing layers. So with more layers and more number of nodes the error is minimum

In [32]:
regr = MLPRegressor(hidden_layer_sizes=(16,16,16,), activation='logistic').fit(X_train_temp, y_train_temp)

lm_y1 = regr.predict(X_train_temp)
lm_y2 = regr.predict(X_test)
lm_y3 = regr.predict(X_val)

print(mean_squared_error(lm_y1,y_train_temp))
print(mean_squared_error(lm_y2,y_test))
print(mean_squared_error(lm_y3,y_val))


0.17013911423639322
0.17820684743120013
0.18698243794679176


 What happens if you change activation functions? Does that matter here?
 - Changing the activation function from 'relu' to 'logistic' for 3 layers 16 nodes each, the error in the training set and test set decreased.